#### This notebook demonstrates how to run QIIME2 on Biowulf for development of pipeline for Nephele

##### (Optional) Update to the latest qiime 2
* wget https://data.qiime2.org/distro/core/qiime2-2020.11-py36-linux-conda.yml
* conda env create -n qiime2-2020.11 --file qiime2-2020.11-py36-linux-conda.yml

##### Step 1: in terminal window 1
```
ssh -X quinonesm@biowulf.nih.gov
source /data/$USER/conda/etc/profile.d/conda.sh
conda activate base
conda activate qiime2-2020.11
jupyter serverextension enable --py qiime2 --sys-prefix
cd /data/quinonesm/
```

##### follow https://hpc.nih.gov/apps/jupyter.html 
```
module load tmux  
tmux  
sinteractive --gres=lscratch:5 --mem=20g --tunnel
```

##### Step 2: in terminal window 2:  open the tunnel per instructions (for example)
```
ssh  -L 38772:localhost:38772 quinonesm@biowulf.nih.gov
```

##### Step 3: back in the terminal window 1
```
module load jupyter && jupyter lab --ip localhost --port $PORT1 --no-browser
```


#### Info for pipeline users: 
##### diagrams: https://docs.qiime2.org/2020.11/tutorials/overview/#let-s-get-oriented-flowcharts  
in particular https://docs.qiime2.org/2020.11/tutorials/overview/#derep-denoise
##### basic tutorial: https://docs.qiime2.org/2020.11/tutorials/moving-pictures/

##### To launch this notebook locally (in Mac), first activate the environment, then pass it to jupyter
```
source /opt/miniconda3/bin/activate* 
conda activate qiime2-2020.11
jupyter serverextension enable --py qiime2 --sys-prefix
jupyter notebook
```
_______

#### The pipeline below will use the same dataset available in the Nephele User Guide <https://nephele.niaid.nih.gov/user_guide/>

In [ ]:
cd /data/quinonesm/Nephele/
qiime info

### Part 1a: Import of demultiplexed PairedEnd reads using indicated filepaths

***The validation of the mapping file will be different for this pipeline because it needs the sample-id and column headers as forward-absolute-filepath, reverse-absolute-filepath.  For single end, it only needs sample-id and absolute-filepath***

***There will be 3 options for import (paired, single and previously joined)***

In [ ]:
# The manifest file (mapping file) must have the absolute path as below.  For the single end, the column header should say "absolute-filepath" instead of "forward-absolute-filepath".
#sample-id	forward-absolute-filepath	reverse-absolute-filepath	Antibiotic	Animal	Day	Tissue
#A22831	$PWD/N2_16S_example_data/22831_S41_R1_subsample.fastq.gz	$PWD/N2_16S_example_data/22831_S41_R2_subsample.fastq.gz	Control	RhDCBC	347	SwabJejunum
#A22833	$PWD/N2_16S_example_data/22833_S45_R1_subsample.fastq.gz	$PWD/N2_16S_example_data/22833_S45_R2_subsample.fastq.gz	Control	RhDCBC	347	SwabRectum
#A22349	$PWD/N2_16S_example_data/22349_S26_R1_subsample.fastq.gz	$PWD/N2_16S_example_data/22349_S26_R2_subsample.fastq.gz	Control	RhDCVf	274	SwabCecum
#A22192	$PWD/N2_16S_example_data/22192_S22_R1_subsample.fastq.gz	$PWD/N2_16S_example_data/22192_S22_R2_subsample.fastq.gz	Vancomycin	RhCL4c	239	SwabRectum
#A22187	$PWD/N2_16S_example_data/22187_S19_R1_subsample.fastq.gz	$PWD/N2_16S_example_data/22187_S19_R2_subsample.fastq.gz	Vancomycin	RhCL4c	239	SwabIleum
#A22061	$PWD/N2_16S_example_data/22061_S5_R1_subsample.fastq.gz	$PWD/N2_16S_example_data/22061_S5_R2_subsample.fastq.gz	Vancomycin	RhDCAV	178	SwabTransverseColon
#A22057	$PWD/N2_16S_example_data/22057_S2_R1_subsample.fastq.gz	$PWD/N2_16S_example_data/22057_S2_R2_subsample.fastq.gz	Vancomycin	RhDCAV	178	SwabJejunum
#A22145	$PWD/N2_16S_example_data/22145_S14_R1_subsample.fastq.gz	$PWD/N2_16S_example_data/22145_S14_R2_subsample.fastq.gz	Control	RhDCKj	239	SwabIleum
#A22350	$PWD/N2_16S_example_data/22350_S27_R1_subsample.fastq.gz	$PWD/N2_16S_example_data/22350_S27_R2_subsample.fastq.gz	Control	RhDCVf	274	SwabTransverseColon
#7pRecSw478.1	$PWD/N2_16S_example_data/23572_S307_R1_subsample.fastq.gz	$PWD/N2_16S_example_data/23572_S307_R2_subsample.fastq.gz	Vancomycin	RhCL7p	478	SwabRectum

##### PairedEnd import

In [ ]:
# import paired end data - sample data from Nephele User Guide.  The mapping file how has a different format of column headers.
qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path N2_16S_mapping_file.txt \
  --output-path paired-end-demux.qza \
  --input-format PairedEndFastqManifestPhred33V2

In [ ]:
# view summary of demultiplexed imported paired end data
qiime demux summarize \
  --i-data paired-end-demux.qza \
  --o-visualization paired-end-demux.qzv

In [ ]:
# join pairs
qiime vsearch join-pairs \
  --i-demultiplexed-seqs paired-end-demux.qza \
  --o-joined-sequences paired-end-joined.qza

In [ ]:
# view summary of joined pairs
qiime demux summarize \
  --i-data paired-end-joined.qza \
  --o-visualization paired-end-joined.qzv

In [ ]:
qiime quality-filter q-score \
  --i-demux  paired-end-joined.qza \
  --p-min-quality 20 \
  --o-filtered-sequences paired-end-joined-filtered.qza \
  --o-filter-stats paired-end-joined-filter-stats.qza

In [ ]:
# view summary of joined pairs and filtered pairs
qiime demux summarize \
  --i-data paired-end-joined-filtered.qza \
  --o-visualization paired-end-joined-filtered.qzv

In [ ]:
# create summary table for paired end
qiime metadata tabulate \
  --m-input-file paired-end-joined-filter-stats.qza \
  --o-visualization paired-end-joined-filter-stats.qzv

In [ ]:
# dereplicate sequences
qiime vsearch dereplicate-sequences \
  --i-sequences paired-end-joined-filtered.qza \
  --o-dereplicated-table table.qza \
  --o-dereplicated-sequences rep-seqs.qza

### Part 1b: Import of demultiplexed SingleEnd reads using indicated filepaths

##### SingleEnd import

In [ ]:
# import - here is the import of Single-end but this should be done for the Paired-End as well
qiime tools import \
  --type 'SampleData[SequencesWithQuality]' \
  --input-path N2_16S_mapping_file_singleend.txt \
  --output-path single-end-demux.qza \
  --input-format SingleEndFastqManifestPhred33V2

In [ ]:
# view summary of demultiplexed
qiime demux summarize \
  --i-data single-end-demux.qza \
  --o-visualization single-end-demux.qzv

In [ ]:
# filter
qiime quality-filter q-score \
  --i-demux  single-end-demux.qza \
  --p-min-quality 20 \
  --o-filtered-sequences single-end-demux-filtered.qza \
  --o-filter-stats single-end-demux-filter-stats.qza

In [ ]:
# dereplicate sequences
qiime vsearch dereplicate-sequences \
  --i-sequences single-end-demux-filtered.qza \
  --o-dereplicated-table table.qza \
  --o-dereplicated-sequences rep-seqs.qza

### Part 1c: Import of demultiplexed prejoined reads using indicated filepaths

In [ ]:
# Note: For pre-joined reads (as in the output of the QC pipe), we will need to add another option
qiime tools import \
  --input-path N2_16S_mapping_file_joined.txt \
  --output-path fj-joined-demux.qza \
  --type SampleData[JoinedSequencesWithQuality] \
  --input-format SingleEndFastqManifestPhred33V2

In [ ]:
# view summary of demultiplexed
qiime demux summarize \
  --i-data fj-joined-demux.qza \
  --o-visualization fj-joined-demux.qzv

In [ ]:
# filter
qiime quality-filter q-score \
  --i-demux  fj-joined-demux.qza \
  --p-min-quality 20 \
  --o-filtered-sequences joined-demux-filtered.qza \
  --o-filter-stats joined-demux-filter-stats.qza

In [ ]:
# dereplicate sequences
qiime vsearch dereplicate-sequences \
  --i-sequences joined-demux-filtered.qza \
  --o-dereplicated-table table.qza \
  --o-dereplicated-sequences rep-seqs.qza

### Part 2: Clustering
***This can be done for both SingleEnd or PairedEnd.  We will give the denovo, closed and open reference options but give the closed as default because it is the lowest compute***

##### Part 2a: Denovo - Let's do 0.97 default for --p-perc-identity

In [ ]:
qiime vsearch cluster-features-de-novo \
  --i-table table.qza \
  --i-sequences rep-seqs.qza \
  --p-perc-identity 0.97 \
  --o-clustered-table table-dn-97.qza \
  --o-clustered-sequences rep-seqs-dn-97.qza

In [ ]:
# create stats and summary of rep-seqs data from denovo clustering
qiime feature-table tabulate-seqs \
--i-data rep-seqs-dn-97.qza \
--o-visualization rep-seqs-dn-97.qzv

##### Import OTU reference sets for clustering

In [ ]:
# import otu tables from greengenes
qiime tools import \
  --input-path /Users/quinonesm/OneDrive_National_Institutes_of_Health/Nephele/qiime2_MiSeq_paired/gg_13_8_otus/rep_set/99_otus.fasta \
  --output-path 99_otus.qza \
  --type 'FeatureData[Sequence]'

# import otu tables
qiime tools import \
  --input-path /Users/quinonesm/OneDrive_National_Institutes_of_Health/Nephele/qiime2_MiSeq_paired/gg_13_8_otus/rep_set/97_otus.fasta \
  --output-path 97_otus.qza \
  --type 'FeatureData[Sequence]'

# import otu tables
qiime tools import \
  --input-path /Users/quinonesm/OneDrive_National_Institutes_of_Health/Nephele/qiime2_MiSeq_paired/gg_13_8_otus/rep_set/85_otus.fasta \
  --output-path 85_otus.qza \
  --type 'FeatureData[Sequence]'

##### Part 2b: Open Reference

In [ ]:
# use 85, 97, or 99 OTU
qiime vsearch cluster-features-open-reference \
  --i-table table.qza \
  --i-sequences rep-seqs.qza \
  --i-reference-sequences 97_otus.qza \
  --p-perc-identity 0.97 \
  --o-clustered-table table-or-97.qza \
  --o-clustered-sequences rep-seqs-or-97.qza \
  --o-new-reference-sequences new-ref-seqs-or-97.qza

In [ ]:
# create stats and summary of rep-seqs data from open-reference clustering
qiime feature-table tabulate-seqs \
--i-data rep-seqs-or-97.qza \
--o-visualization rep-seqs-or-97.qzv

In [ ]:
# rename files
mv rep-seqs-or-97.qza rep-seqs.qza
mv table-or-97.qza table.qza

##### Part 2c: Closed reference

In [ ]:
qiime vsearch cluster-features-closed-reference \
  --i-table table.qza \
  --i-sequences rep-seqs.qza \
  --i-reference-sequences 97_otus.qza \
  --p-perc-identity 0.97 \
  --o-clustered-table table-cr-97.qza \
  --o-clustered-sequences rep-seqs-cr-97.qza \
  --o-unmatched-sequences unmatched-cr-97.qza

In [ ]:
# create stats and summary of rep-seqs data from closed-reference clustering
qiime feature-table tabulate-seqs \
--i-data rep-seqs-cr-97.qza \
--o-visualization rep-seqs-cr-97.qzv

In [ ]:
# rename files
mv rep-seqs-cr-97.qza rep-seqs.qza
mv table-cr-97.qza table.qza

In [ ]:
# summarized denoised sequence variants


### Part 3: Denoising with Deblur
***Perform sequence quality control for Illumina data using the Deblur
  workflow with a 16S reference as a positive filter. Only forward reads are
  supported at this time. The specific reference used is the 88% OTUs from
  Greengenes 13_8. This mode of operation should only be used when data were
  generated from a 16S amplicon protocol on an Illumina platform. The
  reference is only used to assess whether each sequence is likely to be 16S
  by a local alignment using SortMeRNA with a permissive e-value; the
  reference is not used to characterize the sequences.Reference: docs.qiiime2.org)***

In [ ]:
qiime deblur denoise-16S \
  --i-demultiplexed-seqs single-end-demux-filtered.qza \
  --p-trim-length 200 \
  --o-representative-sequences single-end-rep-seqs-deblur.qza \
  --o-table single-end-table-deblur.qza \
  --p-sample-stats \
  --o-stats deblur-single-end-denoised-stats.qza

In [ ]:
# summarize stats from deblur processing
qiime metadata tabulate \
  --m-input-file deblur-single-end-qfilter-stats.qza \
  --o-visualization deblur-single-end-qfilter-stats.qzv
qiime deblur visualize-stats \
  --i-deblur-stats deblur-single-end-denoised-stats.qza \
  --o-visualization deblur-single-end-denoised-stats.qzv

In [ ]:
# summarized denoised sequence variants
qiime feature-table summarize \
  --i-table single-end-table-deblur.qza \
  --o-visualization single-end-table-deblur.qzv \
  --m-sample-metadata-file N2_16S_mapping_file_singleend.txt
qiime feature-table tabulate-seqs \
  --i-data single-end-rep-seqs-deblur.qza \
  --o-visualization single-end-rep-seqs-deblur.qzv

In [ ]:
# rename - This one I didn't run since I wanted to use the open clustering for now
# mv single-end-rep-seqs-deblur.qza rep-seqs.qza
# mv single-end-table-deblur.qza table.qza

### Part 4: Create a phylogenetic tree to be used in downstream diversity steps
***This step will take input from the Denoising steps above (either Deblur or vsearch clustering)***

In [ ]:
# Option A: (default) inferring phylogeny using an reference based fragment insertion approach
# download both files (greengenes and silva) from https://docs.qiime2.org/2020.11/data-resources/#sepp-reference-databases
# sepp-refs-silva-128.qza or sepp-refs-gg-13-8.qza

qiime fragment-insertion sepp \
  --i-representative-sequences rep-seqs.qza \
  --i-reference-database sepp-refs-silva-128.qza \
  --o-tree insertion-tree.qza \
  --p-threads 2 \
  --o-placements insertion-placements.qza

In [ ]:
# Option B: denovo phylogenetic tree  - tested with closed reference
qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences rep-seqs.qza \
  --o-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza \
  --o-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza

### Part 5: Alpha and Beta Diversity - This is similar as the Downstream Analysis pipeline except that it uses the phylogenetic tree

##### Use the sampling depth function as in here https://github.niaid.nih.gov/bcbb/nephele2/blob/next_release/pipelines/DS_analysis_16S/Qiime_2.0_Core_Diversity.md 

##### For the metadata, only a sample ID column is required.  Columns are inferred as numerical or categorical but optionally we could add a line to indicate column type. 
https://docs.google.com/spreadsheets/d/1hBo_NWijLILEFYJrYs7R_Bwc7i_n3ZmPKUQetpVk-pk/edit#gid=0

In [ ]:
# core metrics using phylogeny - similar to what is available in the downstream analysis pipeline.  
rm -rf core-metrics-results_5k_insertion
qiime diversity core-metrics-phylogenetic \
  --i-phylogeny insertion-tree.qza \
  --i-table table.qza \
  --p-sampling-depth 5000 \
  --m-metadata-file N2_16S_mapping_file.txt \
  --output-dir core-metrics-results_5k_insertion

In [ ]:
# core metrics using phylogeny - similar to what is available in the downstream analysis pipeline.  
rm -rf core-metrics-results_5k
qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table table.qza \
  --p-sampling-depth 5000 \
  --m-metadata-file N2_16S_mapping_file.txt \
  --output-dir core-metrics-results_5k

In [ ]:
# alpha diversity group significance
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results_5k/faith_pd_vector.qza \
  --m-metadata-file N2_16S_mapping_file.txt \
  --o-visualization core-metrics-results_5k/faith-pd-group-significance.qzv

qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results_5k/evenness_vector.qza \
  --m-metadata-file N2_16S_mapping_file.txt \
  --o-visualization core-metrics-results_5k/evenness-group-significance.qzv

### Part 6: Alpha rarefaction

In [ ]:
qiime diversity alpha-rarefaction \
  --i-table table.qza \
  --i-phylogeny rooted-tree.qza \
  --p-max-depth 10000 \
  --m-metadata-file N2_16S_mapping_file.txt \
  --o-visualization alpha-rarefaction.qzv

### Part 7: Taxonomy classification

In [ ]:
# get the classifiers (Silva and Greengenes) - These are the ones trained only to the V4 region.
#Naive Bayes classifiers trained on:
#Silva 138 99% OTUs full-length sequences (MD5: fddefff8bfa2bbfa08b9cad36bcdf709)
#Silva 138 99% OTUs from 515F/806R region of sequences (MD5: 28105eb0f1256bf38b9bb310c701dc4e)
#Greengenes 13_8 99% OTUs full-length sequences (MD5: 03078d15b265f3d2d73ce97661e370b1)
#Greengenes 13_8 99% OTUs from 515F/806R region of sequences (MD5: 682be39339ef36a622b363b8ee2ff88b)
# we could make additional classifiers for other 16S regions or the entire 16S

# classifier for full 16S using OTUs 99%
# wget https://data.qiime2.org/2020.11/common/silva-138-99-nb-classifier.qza
# wget https://data.qiime2.org/2020.11/common/gg-13-8-99-nb-classifier.qza

# classifier for V4 region using OTUs 99%
#wget https://data.qiime2.org/2020.11/common/gg-13-8-99-515-806-nb-classifier.qza
#wget https://data.qiime2.org/2020.11/common/silva-138-99-515-806-nb-classifier.qza

# (optional) - make a classifier by following https://docs.qiime2.org/2020.11/tutorials/feature-classifier/ 

In [ ]:
# First allow user to select option to classify (sklearn or vsearch), then allow selection of classifier (silva or greengenes).
# The reads-per-batch allow for faster processing

In [ ]:
# classify below (tested on biowulf interactive session as: sinteractive --mem=20g --cpus-per-task=4)

##### Part 7a: Classify method sklearn

In [ ]:
# Select to classify with (slow: full length or faster: only for V4 region)
# option 1: SILVA V4 region
qiime feature-classifier classify-sklearn \
  --i-reads rep-seqs.qza \
  --i-classifier silva-138-99-515-806-nb-classifier.qza \
  --o-classification taxonomy.qza \
  --p-reads-per-batch 100

# option 2: Greengenes V4 region
qiime feature-classifier classify-sklearn \
  --i-reads rep-seqs.qza \
  --i-classifier gg-13-8-99-515-806-nb-classifier.qza \
  --o-classification taxonomy.qza \
  --p-reads-per-batch 100

# option 3: SILVA full length
qiime feature-classifier classify-sklearn \
  --i-reads rep-seqs.qza \
  --i-classifier silva-138-99-nb-classifier.qza \
  --o-classification taxonomy.qza \
  --p-reads-per-batch 100

# option 4: Greengenes full length
qiime feature-classifier classify-sklearn \
  --i-reads rep-seqs.qza \
  --i-classifier gg-13-8-99-nb-classifier.qza \
  --o-classification taxonomy.qza \
  --p-reads-per-batch 100

##### Part 7b:Classify method vsearch

In [ ]:
# two options (silva and greengenes)

qiime feature-classifier classify-consensus-vsearch \
  --i-query rep-seqs.qza \
  --i-reference-reads silva-138-99-seqs.qza \
  --i-reference-taxonomy silva-138-99-tax.qza \
  --o-classification taxonomy.qza

In [ ]:
# view taxonomy assignments
qiime metadata tabulate \
  --m-input-file taxonom.qza \
  --o-visualization taxonomy.qzv

### Part 8:Make Barplots

In [ ]:
# make table of rep sequences
qiime feature-table tabulate-seqs \
  --i-data rep-seqs.qza \
  --o-visualization rep_set.qzv

In [ ]:
# start by filtering or subsample the table to 2000 for example
qiime feature-table filter-samples \
  --i-table table.qza \
  --p-min-frequency 2000 \
  --o-filtered-table table_2k.qza

In [ ]:
# make barplot
qiime taxa barplot \
  --i-table table_2k.qza \
  --i-taxonomy taxonomy.qza \
  --m-metadata-file N2_16S_mapping_file.txt \
  --o-visualization taxa_barplot.qzv

___________

#### Part 9: Optional analyses - Let's not worry about this yet
***(I wonder if these can be done here or in the downstream analysis pipeline).  It will need metadata file columns headers to be presented to the user for the user to select from.  For each of the metadata colums, the beta-group-significance function will be run.***

In [ ]:
# beta group significance - This will be an optional step
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results_5k/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file N2_16S_mapping_file.txt \
  --m-metadata-column Antibiotic \
  --o-visualization core-metrics-results_5k/unweighted-unifrac-Antibiotic-significance.qzv \
  --p-pairwise

In [ ]:
# permanova for exploring large differences between groups even with large within group variances
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results_5k/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file N2_16S_mapping_file.txt \
  --m-metadata-column Antibiotic \
  --o-visualization core-metrics-results_5k/unweighted-unifrac-Antibiotic-significance_disp.qzv \
  --p-method permdisp

In [ ]:
# adonis
qiime diversity adonis \
  --i-distance-matrix core-metrics-results_5k/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file N2_16S_mapping_file.txt \
  --o-visualization core-metrics-results_5k/unweighted_Antibiotic+Animal_adonis.qzv \
  --p-formula Antibiotic+Animal

In [ ]:
#Other analyses that can be added:
#1) To determine if the continuous sample metadata is correlated with sample composition, an association test can be run using:
#qiime metadata distance-matrix in combination with qiime diversity mantel and qiime diversity bioenv commands.
#2) differential abundance

In [ ]:
# create stats and summary of rep-seqs data from denovo clustering
qiime feature-table tabulate-seqs \
--i-data rep-seqs-dn-99.qza \
--o-visualization rep-seqs-dn-99.qzv